In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [3]:
df = pd.read_csv("drive/MyDrive/aug_train.csv")
df.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0
3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1
4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0


In [4]:
df = df.drop("enrollee_id", axis=1)

In [5]:
df['gender'] = df['gender'].fillna('Unknown')
df['enrolled_university'] = df['enrolled_university'].fillna('Unknown')
df['education_level'] = df['education_level'].fillna('Unknown')
df['major_discipline'] = df['major_discipline'].fillna('Unknown')
df['experience'] = df['experience'].fillna('Unknown')
df['company_size'] = df['company_size'].fillna('Unknown')
df['company_type'] = df['company_type'].fillna('Unknown')
df['last_new_job'] = df['last_new_job'].fillna('Unknown')

In [6]:
label_cols = [
    'gender','relevent_experience','enrolled_university',
    'education_level','major_discipline','experience',
    'company_size','company_type','last_new_job','city'
]

for col in label_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

In [7]:
scaler = StandardScaler()
df[['city_development_index', 'training_hours']] = scaler.fit_transform(
    df[['city_development_index', 'training_hours']]
)

In [8]:
X = df.drop("target", axis=1)
y = df["target"]

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Training shape:", X_train.shape)
print("Testing shape:", X_test.shape)

Training shape: (15326, 12)
Testing shape: (3832, 12)


In [10]:
# Save full preprocessed dataset
df.to_csv("preprocessed_full.csv", index=False)
print("✅ Preprocessed full dataset saved as preprocessed_full.csv")

# Save training set
train_data = X_train.copy()
train_data["target"] = y_train
train_data.to_csv("train_preprocessed.csv", index=False)
print("✅ Training set saved as train_preprocessed.csv")

# Save test set
test_data = X_test.copy()
test_data["target"] = y_test
test_data.to_csv("test_preprocessed.csv", index=False)
print("✅ Test set saved as test_preprocessed.csv")

✅ Preprocessed full dataset saved as preprocessed_full.csv
✅ Training set saved as train_preprocessed.csv
✅ Test set saved as test_preprocessed.csv


In [ ]:
u

In [11]:
import xgboost as xgb

# Initialize XGBoost Classifier
xgb_model = xgb.XGBClassifier(random_state=42)

# Train the model
xgb_model.fit(X_train, y_train)

print("✅ XGBoost model trained successfully!")

✅ XGBoost model trained successfully!


In [12]:
# Make predictions on the test set
y_pred = xgb_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print("\nConfusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", class_report)

# Save the trained model
import joblib
joblib.dump(xgb_model, "xgb_model.pkl")
print("\n✅ Trained XGBoost model saved as xgb_model.pkl")

Accuracy: 0.7847

Confusion Matrix:
 [[2528  352]
 [ 473  479]]

Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.88      0.86      2880
           1       0.58      0.50      0.54       952

    accuracy                           0.78      3832
   macro avg       0.71      0.69      0.70      3832
weighted avg       0.78      0.78      0.78      3832


✅ Trained XGBoost model saved as xgb_model.pkl
